In [1]:
import requests, json, pprint
import pandas as pd
from jsonpath_ng import jsonpath, parse


In [ ]:
pip install jsonpath_ng
pip install pandas

In [7]:
#1.load input 
timestamp_csv = pd.read_csv ('timestamp1.csv')
coordinate_csv = pd.read_csv('coordinate.csv')
# API_KEY = "69e792e1189be59d0196f827035b39cb" #alternative api key not yet activated
API_KEY = "d8a7f0f4a549a5083e601317b765430f"
units = "metric"

unixtodatetime_list=[]
datetime_list=[]
temp_list=[]
humidity_list=[]

k=0

In [8]:
while k < len(coordinate_csv. index):

    location = coordinate_csv.iloc[k]['Name']
    lat = str(coordinate_csv.iloc[k]['lat'])
    lon = str(coordinate_csv.iloc[k]['lon'])
    
    print(location)
    print(lat)
    print(lon)
    
    i=0

    while i < len(timestamp_csv. index):  #TO-DO, if x datapoint to request, i< x.

        dt = str(timestamp_csv.loc[i].at["Timestamp"])

        #2.API Request & Response

        URL = "https://api.openweathermap.org/data/3.0/onecall/timemachine?" + "lat=" + lat + "&lon=" + lon + "&dt=" + dt + "&appid=" + API_KEY + "&units=" + units
        response = requests.get(URL)

        # checking the status code of the request
        
        if response.status_code == 200:
        # getting data in the json format
            data = response.json()
         #getting required info
            uplat = parse("$.lat").find(data)[0].value
            uplon = parse("$.lon").find(data)[0].value
            datetime= parse("$.data[*].dt").find(data)[0].value
            temperature = parse("$.data[*].temp").find(data)[0].value
            humidity = parse("$.data[*].humidity").find(data)[0].value
            unixtodatetime = pd.to_datetime(dt, unit="s")
            unixtodatime_converted= str(unixtodatetime.tz_localize('UTC').tz_convert('Asia/Manila'))
            

                # print(f"lat: {uplat}")
                # print(f"lon: {uplon}")
                # print(f"datetime: {datetime}")
                # print(f"Temperature: {temperature}")
                # print(f"Humidity: {humidity}")

            i=i+1

        else:
            # showing the error message
            print("Error in the HTTP request")

        #3. Write data into dataframe
        unixtodatetime_list.append(unixtodatime_converted)
        datetime_list.append(datetime)
        temp_list.append(temperature)
        humidity_list.append(humidity)

    else:
        d = {'timestamp':datetime_list,'date_time':unixtodatetime_list,'temperature':temp_list,'humidity':humidity_list}
        df = pd.DataFrame(d)
        df.to_csv(location + '.csv')
        print("Location:" + location + " done.\n")
        
        #Reset the dataframe for next iteration
        unixtodatetime_list=[]
        datetime_list=[]
        temp_list=[]
        humidity_list=[]
        
    k=k+1
    
else:
    print("All data logged succesfully, check the output.")

    

Arsen
7.319442714
125.6933521
Location:Arsen done.

Edith
14.63198461
121.0347292
Location:Edith done.

Charline
8.593098214
123.3645233
Location:Charline done.

Betg
15.5146381
120.8900767
Location:Betg done.

Rose
14.44056891
120.9996015
Location:Rose done.

Jonai
14.59635467
121.0974235
Location:Jonai done.

Roid
14.66292438
120.9607268
Location:Roid done.

Ren
14.72152235
120.5445843
Location:Ren done.

All data logged succesfully, check the output.
